In [1]:
import datetime
import random
import json
import dotenv
from azure.iot.device import IoTHubDeviceClient
from azure.iot.hub import IoTHubRegistryManager
from os import getenv

In [2]:
oid = "XYZ"

In [3]:
PEDOMETER_DEVICE_CONNECTION_STRING = "CONNECTION_STRING"
pedometer_device_id = PEDOMETER_DEVICE_CONNECTION_STRING.split(";")[1].split("=")[1]

In [4]:
SCALE_DEVICE_CONNECTION_STRING = "CONNECTION_STRING"
scale_device_id = SCALE_DEVICE_CONNECTION_STRING.split(";")[1].split("=")[1]

In [5]:
dotenv.load_dotenv("./device-function-app/.env")
# pRINT OUT THE ENVIRONMENT VARIABLES
getenv("IOT_HUB_CONNECTION_STRING")

'HostName=my-project-iot-hub.azure-devices.net;SharedAccessKeyName=iothubowner;SharedAccessKey=6ccIWRhjQzmZz6Lu6Tzm8jmIB7+F5Fg2xAIoTIrXKNI='

In [6]:
registry = IoTHubRegistryManager.from_connection_string(getenv("IOT_HUB_CONNECTION_STRING"))

In [7]:
pedometer_device_twin = registry.get_twin(pedometer_device_id)

In [ ]:
# Create an IoT Hub client
pedometer_device = IoTHubDeviceClient.create_from_connection_string(PEDOMETER_DEVICE_CONNECTION_STRING)
pedometer_device.connect()
scale_device = IoTHubDeviceClient.create_from_connection_string(SCALE_DEVICE_CONNECTION_STRING)
scale_device.connect()

ConnectionStateStage: DisconnectEvent received while in unexpected state - ConnectionState.DISCONNECTED
Exception caught in background thread.  Unable to handle.
['azure.iot.device.common.transport_exceptions.ConnectionDroppedError: Unexpected disconnection\n']
Exception caught in background thread.  Unable to handle.
['azure.iot.device.common.transport_exceptions.ConnectionDroppedError: Unexpected disconnection\n']
ConnectionStateStage: DisconnectEvent received while in unexpected state - ConnectionState.DISCONNECTED
Exception caught in background thread.  Unable to handle.
['azure.iot.device.common.transport_exceptions.ConnectionDroppedError: Unexpected disconnection\n']
Exception caught in background thread.  Unable to handle.
['azure.iot.device.common.transport_exceptions.ConnectionDroppedError: Unexpected disconnection\n']


In [9]:
from abc import ABC
class Measurement(ABC):
    def __init__(self, oid: str, datetime: str):
        self.oid = oid
        self.datetime = datetime

In [10]:
class PedometerMeasurement(Measurement):
    def __init__(self, steps, distance, datetime: datetime.datetime, oid: str, unit="30m"):
        super().__init__(oid, datetime.isoformat())
        self.steps = steps
        self.distance = distance
        self.unit = unit
    
    def __str__(self):
        return f"Device Type: Steps: {self.steps}, Distance: {self.distance}, Unit: {self.unit}, Datetime: {self.datetime}"

In [11]:
class ScaleMeasurement(Measurement):
    def __init__(self, weight, timestamp: datetime.datetime, oid: str, unit="kg"):
        super().__init__(oid, timestamp.isoformat())
        self.weight = weight
        self.unit = unit

    def __str__(self):
        return f"Device Type: Weight: {self.weight}, Unit: {self.unit}, Datetime: {self.datetime}"

In [12]:
current_time = datetime.datetime.now()
current_time -= datetime.timedelta(minutes=current_time.minute % 30, seconds=current_time.second, microseconds=current_time.microsecond)
thirty_minutes = datetime.timedelta(minutes=45) # Round to the previous whole 30 minutes
# Send pedometer measurements
for i in range(300):
    pm = PedometerMeasurement(random.randint(0, 100), random.randint(0, 100), current_time, oid)
    print(pm)
    current_time -= thirty_minutes
    message = json.dumps(pm.__dict__)
    print(message)
    pedometer_device.send_message(message)

Device Type: Steps: 76, Distance: 97, Unit: 30m, Datetime: 2024-02-10T13:00:00
{"oid": "XYZ", "datetime": "2024-02-10T13:00:00", "steps": 76, "distance": 97, "unit": "30m"}
Device Type: Steps: 21, Distance: 7, Unit: 30m, Datetime: 2024-02-10T12:15:00
{"oid": "XYZ", "datetime": "2024-02-10T12:15:00", "steps": 21, "distance": 7, "unit": "30m"}
Device Type: Steps: 0, Distance: 17, Unit: 30m, Datetime: 2024-02-10T11:30:00
{"oid": "XYZ", "datetime": "2024-02-10T11:30:00", "steps": 0, "distance": 17, "unit": "30m"}
Device Type: Steps: 28, Distance: 87, Unit: 30m, Datetime: 2024-02-10T10:45:00
{"oid": "XYZ", "datetime": "2024-02-10T10:45:00", "steps": 28, "distance": 87, "unit": "30m"}
Device Type: Steps: 91, Distance: 18, Unit: 30m, Datetime: 2024-02-10T10:00:00
{"oid": "XYZ", "datetime": "2024-02-10T10:00:00", "steps": 91, "distance": 18, "unit": "30m"}
Device Type: Steps: 66, Distance: 37, Unit: 30m, Datetime: 2024-02-10T09:15:00
{"oid": "XYZ", "datetime": "2024-02-10T09:15:00", "steps": 6

In [13]:
current_time = datetime.datetime.now()
current_time -= datetime.timedelta(minutes=current_time.minute % 30, seconds=current_time.second, microseconds=current_time.microsecond)
thirty_minutes = datetime.timedelta(minutes=45) # Round to the previous whole 30 minutes
# Send pedometer measurements
for i in range(300):
    pm = ScaleMeasurement(random.randint(65, 75) + random.random(), current_time, oid)
    print(pm)
    current_time -= thirty_minutes
    message = json.dumps(pm.__dict__)
    print(message)
    scale_device.send_message(message)

Device Type: Weight: 74.89430621515532, Unit: kg, Datetime: 2024-02-10T13:00:00
{"oid": "XYZ", "datetime": "2024-02-10T13:00:00", "weight": 74.89430621515532, "unit": "kg"}
Device Type: Weight: 73.1919451188054, Unit: kg, Datetime: 2024-02-10T12:15:00
{"oid": "XYZ", "datetime": "2024-02-10T12:15:00", "weight": 73.1919451188054, "unit": "kg"}
Device Type: Weight: 75.16944149054379, Unit: kg, Datetime: 2024-02-10T11:30:00
{"oid": "XYZ", "datetime": "2024-02-10T11:30:00", "weight": 75.16944149054379, "unit": "kg"}
Device Type: Weight: 74.54281444323514, Unit: kg, Datetime: 2024-02-10T10:45:00
{"oid": "XYZ", "datetime": "2024-02-10T10:45:00", "weight": 74.54281444323514, "unit": "kg"}
Device Type: Weight: 66.2383585589296, Unit: kg, Datetime: 2024-02-10T10:00:00
{"oid": "XYZ", "datetime": "2024-02-10T10:00:00", "weight": 66.2383585589296, "unit": "kg"}
Device Type: Weight: 71.68863493440075, Unit: kg, Datetime: 2024-02-10T09:15:00
{"oid": "XYZ", "datetime": "2024-02-10T09:15:00", "weight":